# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 24 2022 10:55AM,249246,19,MSP215149,"Methapharm, Inc.",Released
1,Oct 24 2022 10:52AM,249245,19,MSP215146,"Methapharm, Inc.",Released
2,Oct 24 2022 10:46AM,249240,15,30011624,"Alliance Pharma, Inc.",Executing
3,Oct 24 2022 10:46AM,249240,15,30011625,"Alliance Pharma, Inc.",Executing
4,Oct 24 2022 10:46AM,249240,15,30011626,"Alliance Pharma, Inc.",Released
5,Oct 24 2022 10:46AM,249240,15,30011627,"Alliance Pharma, Inc.",Released
6,Oct 24 2022 10:46AM,249240,15,30011628,"Alliance Pharma, Inc.",Executing
7,Oct 24 2022 10:46AM,249240,15,30011629,"Alliance Pharma, Inc.",Released
8,Oct 24 2022 10:46AM,249240,15,30011630,"Alliance Pharma, Inc.",Released
9,Oct 24 2022 10:46AM,249240,15,30011631,"Alliance Pharma, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,249241,Released,27
26,249242,Released,1
27,249244,Released,1
28,249245,Released,1
29,249246,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249241,NaN,NaN,27.0
249242,NaN,NaN,1.0
249244,NaN,NaN,1.0
249245,NaN,NaN,1.0
249246,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249209,1.0,15.0,3.0
249212,0.0,8.0,2.0
249216,0.0,0.0,1.0
249218,0.0,0.0,1.0
249219,0.0,0.0,57.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249209,1,15,3
249212,0,8,2
249216,0,0,1
249218,0,0,1
249219,0,0,57


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249209,1,15,3
1,249212,0,8,2
2,249216,0,0,1
3,249218,0,0,1
4,249219,0,0,57


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249209,1,15,3
1,249212,,8,2
2,249216,,,1
3,249218,,,1
4,249219,,,57


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 24 2022 10:55AM,249246,19,"Methapharm, Inc."
1,Oct 24 2022 10:52AM,249245,19,"Methapharm, Inc."
2,Oct 24 2022 10:46AM,249240,15,"Alliance Pharma, Inc."
17,Oct 24 2022 10:41AM,249244,12,HealthCaps LLC
18,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC"
45,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc."
46,Oct 24 2022 10:25AM,249237,10,ISDIN Corporation
71,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation
73,Oct 24 2022 10:09AM,249235,10,ISDIN Corporation
74,Oct 24 2022 10:09AM,249233,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 24 2022 10:55AM,249246,19,"Methapharm, Inc.",,,1
1,Oct 24 2022 10:52AM,249245,19,"Methapharm, Inc.",,,1
2,Oct 24 2022 10:46AM,249240,15,"Alliance Pharma, Inc.",,7,8
3,Oct 24 2022 10:41AM,249244,12,HealthCaps LLC,,,1
4,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",,,27
5,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc.",,,1
6,Oct 24 2022 10:25AM,249237,10,ISDIN Corporation,,,25
7,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation,,,2
8,Oct 24 2022 10:09AM,249235,10,ISDIN Corporation,,,1
9,Oct 24 2022 10:09AM,249233,10,Bio-PRF,,2,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 10:55AM,249246,19,"Methapharm, Inc.",1,,
1,Oct 24 2022 10:52AM,249245,19,"Methapharm, Inc.",1,,
2,Oct 24 2022 10:46AM,249240,15,"Alliance Pharma, Inc.",8,7,
3,Oct 24 2022 10:41AM,249244,12,HealthCaps LLC,1,,
4,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",27,,
5,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc.",1,,
6,Oct 24 2022 10:25AM,249237,10,ISDIN Corporation,25,,
7,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation,2,,
8,Oct 24 2022 10:09AM,249235,10,ISDIN Corporation,1,,
9,Oct 24 2022 10:09AM,249233,10,Bio-PRF,12,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 10:55AM,249246,19,"Methapharm, Inc.",1,,
1,Oct 24 2022 10:52AM,249245,19,"Methapharm, Inc.",1,,
2,Oct 24 2022 10:46AM,249240,15,"Alliance Pharma, Inc.",8,7,
3,Oct 24 2022 10:41AM,249244,12,HealthCaps LLC,1,,
4,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",27,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 10:55AM,249246,19,"Methapharm, Inc.",1,NaN,NaN
1,Oct 24 2022 10:52AM,249245,19,"Methapharm, Inc.",1,NaN,NaN
2,Oct 24 2022 10:46AM,249240,15,"Alliance Pharma, Inc.",8,7.0,NaN
3,Oct 24 2022 10:41AM,249244,12,HealthCaps LLC,1,NaN,NaN
4,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",27,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 10:55AM,249246,19,"Methapharm, Inc.",1,0.0,0.0
1,Oct 24 2022 10:52AM,249245,19,"Methapharm, Inc.",1,0.0,0.0
2,Oct 24 2022 10:46AM,249240,15,"Alliance Pharma, Inc.",8,7.0,0.0
3,Oct 24 2022 10:41AM,249244,12,HealthCaps LLC,1,0.0,0.0
4,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",27,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3987586,352,33.0,1.0
12,249244,1,0.0,0.0
15,498481,35,7.0,0.0
16,249231,1,0.0,0.0
19,996949,8,0.0,0.0
20,249242,1,0.0,0.0
21,249234,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3987586,352,33.0,1.0
1,12,249244,1,0.0,0.0
2,15,498481,35,7.0,0.0
3,16,249231,1,0.0,0.0
4,19,996949,8,0.0,0.0
5,20,249242,1,0.0,0.0
6,21,249234,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,352,33.0,1.0
1,12,1,0.0,0.0
2,15,35,7.0,0.0
3,16,1,0.0,0.0
4,19,8,0.0,0.0
5,20,1,0.0,0.0
6,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,352.0
1,12,Released,1.0
2,15,Released,35.0
3,16,Released,1.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,33.0,0.0,7.0,0.0,0.0,0.0,0.0
Released,352.0,1.0,35.0,1.0,8.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,33.0,0.0,7.0,0.0,0.0,0.0,0.0
2,Released,352.0,1.0,35.0,1.0,8.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,33.0,0.0,7.0,0.0,0.0,0.0,0.0
2,Released,352.0,1.0,35.0,1.0,8.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()